#  Discipline analysis of grant proposals

It is important for a granting agency to know how the distribution of the applications qua disciplines is. 
ˆ How many applications belong to Exact Science disciplines, how many fall within one discipline?
ˆ How many applications with disciplines outside exact sciences domain have been submitted?

reference

https://github.com/RaThorat/discipline_analyses_of_proposals

Importing various modules required for analysis

In [1]:
import xlsxwriter 
import scipy.sparse as sp
import numpy as np
import os
os.getcwd()
import tkinter as tk
from tkinter import filedialog
import pandas as pd

Importing the submitted proposals file with Tkinter module, 'Example input file submitted proposals.xlsx'

In [2]:
root= tk.Tk()

canvas1 = tk.Canvas(root, width = 600, height = 300, bg = 'lightsteelblue')
canvas1.pack()

def getExcel ():
    global dfA
    
    import_file_path = filedialog.askopenfilename()
    dfA = pd.read_excel (import_file_path)
    #print(dfA)
    
browseButton_Excel = tk.Button(text='Import "Example input file submitted proposals.xlsx"', command=getExcel, bg='green', fg='white', font=('helvetica', 12, 'bold'))
canvas1.create_window(200, 200, window=browseButton_Excel)

root.mainloop()


Importing the successful proposals file with Tkinter module

In [3]:
root= tk.Tk()

canvas1 = tk.Canvas(root, width = 600, height = 300, bg = 'lightsteelblue')
canvas1.pack()

def getExcel ():
    global dfG
    
    import_file_path = filedialog.askopenfilename()
    dfG = pd.read_excel (import_file_path)
    #print (dfG)
    
browseButton_Excel = tk.Button(text='Import "Example input file successful proposals.xlsx"', command=getExcel, bg='green', fg='white', font=('helvetica', 12, 'bold'))
canvas1.create_window(200, 200, window=browseButton_Excel)

root.mainloop()


## Cleaning of the datafames

Turning the NaN values in the dataframes into zero values

In [4]:
dfA=dfA.fillna(0)
dfG=dfG.fillna(0)

checking the excel file in the form of data frame

In [5]:
dfA.head(2)

,Application numbers,Earth sciences,Archeology,Biology,Animal Welfare,Gender studies,Medicine,Information Technology,Art and Architecture,Life sciences,...,Enviormental Sciences,Physics,Physics and Technology,Development studies,Psychology,Chemistry,Astronomy,Language,Technical Sciences,Mathematics
0,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [6]:
len(dfA.index)

92

checking how many rows and columns the dataframe has

In [7]:
dfA.shape

(92, 21)

checking the excel file in the form of data frame

In [8]:
dfG.head(2)

,Application numbers,Earth sciences,Archeology,Biology,Animal Welfare,Gender studies,Medicine,Information Technology,Art and Architecture,Life sciences,...,Enviormental Sciences,Physics,Physics and Technology,Development studies,Psychology,Chemistry,Astronomy,Language,Technical Sciences,Mathematics
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


checking how many rows and columns the dataframe has

In [9]:
dfG.shape

(92, 21)

Adding a column for counting all discipline appearances per proposal

In [10]:
dfA['All_Discipline_count']=dfA.loc[:,'Earth sciences' : 'Mathematics'].sum(axis=1)
dfG['All_Discipline_count']=dfG.loc[:,'Earth sciences' : 'Mathematics'].sum(axis=1)

Adding a column ENWtotaal for counting Exact and Natural sciences (ENW) discipline appearances per proposal

In [11]:
dfA['ENW_Discipline_count']=dfA.loc[:, ['Earth sciences', 'Biology', 'Information Technology','Life sciences', 'Enviormental Sciences', 'Physics', 'Chemistry', 'Astronomy', 'Mathematics']].sum(axis=1)
dfG['ENW_Discipline_count']=dfG.loc[:, ['Earth sciences', 'Biology', 'Information Technology','Life sciences', 'Enviormental Sciences', 'Physics', 'Chemistry', 'Astronomy', 'Mathematics']].sum(axis=1)


Segregating multidiscipline proposals from monodiscipline proposals in apart dataframe

In [12]:
dfA_multidisci=dfA[(dfA.All_Discipline_count>1)]
dfG_multidisci=dfG[(dfA.All_Discipline_count>1)]

Segregating ENW monodiscipline proposals from non ENW monodiscipline proposals in apart dataframe

In [13]:
dfA_ENW_monodisci=dfA[(dfA.All_Discipline_count==1) & (dfA.ENW_Discipline_count==1)]
dfG_ENW_monodisci=dfA[(dfG.All_Discipline_count==1) & (dfG.ENW_Discipline_count==1)]

Segregating non ENW monodiscipline proposals from non ENW monodiscipline proposals in apart dataframe

In [14]:
dfA_zonder_ENW_monodisci=dfA[(dfA.All_Discipline_count==1) & (dfA.ENW_Discipline_count==0)]#selecting the non ENW monodiscipline rows
dfG_zonder_ENW_monodisci=dfG[(dfG.All_Discipline_count==1) & (dfG.ENW_Discipline_count==0)]#selecting the non ENW monodiscipline rows

checking multidiscipline proposals dataframe

In [15]:
dfA_multidisci.head(2)

,Application numbers,Earth sciences,Archeology,Biology,Animal Welfare,Gender studies,Medicine,Information Technology,Art and Architecture,Life sciences,...,Physics and Technology,Development studies,Psychology,Chemistry,Astronomy,Language,Technical Sciences,Mathematics,All_Discipline_count,ENW_Discipline_count
1,2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,3.0
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,2.0


checking ENW monodiscipline proposals dataframe

In [16]:
dfA_ENW_monodisci.head(2)

,Application numbers,Earth sciences,Archeology,Biology,Animal Welfare,Gender studies,Medicine,Information Technology,Art and Architecture,Life sciences,...,Physics and Technology,Development studies,Psychology,Chemistry,Astronomy,Language,Technical Sciences,Mathematics,All_Discipline_count,ENW_Discipline_count
0,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


checking non ENW monodiscipline proposals dataframe

In [17]:
dfA_zonder_ENW_monodisci.head(2)

,Application numbers,Earth sciences,Archeology,Biology,Animal Welfare,Gender studies,Medicine,Information Technology,Art and Architecture,Life sciences,...,Physics and Technology,Development studies,Psychology,Chemistry,Astronomy,Language,Technical Sciences,Mathematics,All_Discipline_count,ENW_Discipline_count
46,47,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


Segregating interdomein multidiscipline proposals (containing ENW disciplines) in apart dataframe

In [18]:
dfA_interdom_multidisci=dfA.loc[(dfA.All_Discipline_count>1) & (dfA.All_Discipline_count!=dfA.ENW_Discipline_count) & (dfA.ENW_Discipline_count!=0)]
dfG_interdom_multidisci=dfG.loc[(dfG.All_Discipline_count>1) & (dfG.All_Discipline_count!=dfG.ENW_Discipline_count)& (dfG.ENW_Discipline_count!=0)]

Segregating intradomein multidiscipline proposals (containing ENW disciplines) in apart dataframe

In [19]:
dfA_intradom_multidisci=dfA.loc[(dfA.All_Discipline_count>1) & (dfA.All_Discipline_count==dfA.ENW_Discipline_count)]
dfG_intradom_multidisci=dfG.loc[(dfG.All_Discipline_count>1) & (dfG.All_Discipline_count==dfG.ENW_Discipline_count)]

Segregating without ENW multidiscipline proposals in apart dataframe

In [20]:
dfA_zonder_ENW_multidisci=dfA.loc[(dfA.All_Discipline_count>1) & (dfA.ENW_Discipline_count==0)]
dfG_zonder_ENW_multidisci=dfG.loc[(dfG.All_Discipline_count>1) & (dfG.ENW_Discipline_count==0)]

## creation of a summary dataframe

In [21]:
df_summary = pd.DataFrame(columns = ['Application character', 'Submitted', 'Successful'])
df_summary['Application character']=['ENW monodisci', 'ENW intradomein multidisci','ENW interdomein multidisci','other monodisci', 'other multidisci', 'Total']
df_summary['Submitted']=[dfA_ENW_monodisci.ENW_Discipline_count.count(), dfA_intradom_multidisci.All_Discipline_count.count(), dfA_interdom_multidisci.All_Discipline_count.count(), dfA_zonder_ENW_monodisci.All_Discipline_count.count(), dfA_zonder_ENW_multidisci.All_Discipline_count.count(), dfA_ENW_monodisci.ENW_Discipline_count.count()+ dfA_intradom_multidisci.All_Discipline_count.count()+ dfA_interdom_multidisci.All_Discipline_count.count()+ dfA_zonder_ENW_monodisci.All_Discipline_count.count()+ dfA_zonder_ENW_multidisci.All_Discipline_count.count()]
df_summary['Successful']=[dfG_ENW_monodisci.ENW_Discipline_count.count(), dfG_intradom_multidisci.All_Discipline_count.count(), dfG_interdom_multidisci.All_Discipline_count.count(), dfG_zonder_ENW_monodisci.All_Discipline_count.count(), dfG_zonder_ENW_multidisci.All_Discipline_count.count(), dfG_ENW_monodisci.ENW_Discipline_count.count()+dfG_intradom_multidisci.All_Discipline_count.count()+dfG_interdom_multidisci.All_Discipline_count.count()+ dfG_zonder_ENW_monodisci.All_Discipline_count.count()+ dfG_zonder_ENW_multidisci.All_Discipline_count.count()]

#adding a column of % Succes
df_summary['% Succes']=df_summary['Successful'].div(df_summary['Submitted'].values, axis=0)
df_summary['% Succes']=df_summary['% Succes'].mul(100).round(1)
#check the summary dataframe
df_summary

,Application character,Submitted,Successful,% Succes
0,ENW monodisci,45,12,26.7
1,ENW intradomein multidisci,40,7,17.5
2,ENW interdomein multidisci,6,1,16.7
3,other monodisci,1,0,0.0
4,other multidisci,0,0,NaN
5,Total,92,20,21.7


Dropping the columns unncessary for next steps data processing

In [22]:
dfA_clean_multidisci=dfA_multidisci.drop(columns=['Application numbers','All_Discipline_count', 'ENW_Discipline_count'])
dfG_clean_multidisci=dfG_multidisci.drop(columns=['Application numbers','All_Discipline_count', 'ENW_Discipline_count'])

## Creation of co-occurance matrix for submitted proposals

In [23]:
cols = dfA_clean_multidisci.columns
XA = sp.csr_matrix(dfA_clean_multidisci.astype(int).values)
XcA = XA.T * XA  # multiply sparse matrix
XcA.setdiag(0)  # reset diagonal

# create dataframe from co-occurence matrix in dense format
dfA_co_occ_mtx = pd.DataFrame(XcA.todense(), index=cols, columns=cols)

/home/gebruiker/anaconda3/lib/python3.9/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


## Creation of co-occurance matrix for successful proposals

In [24]:
cols = dfG_clean_multidisci.columns
XG = sp.csr_matrix(dfG_clean_multidisci.astype(int).values)
XcG = XG.T * XG  # multiply sparse matrix
XcG.setdiag(0)  # reset diagonal

# create dataframe from co-occurence matrix in dense format
dfG_co_occ_mtx = pd.DataFrame(XcG.todense(), index=cols, columns=cols)

## create co occurance matrices in a format for VOSviewer

for submitted proposals

In [25]:
dfA_netwerk = dfA_co_occ_mtx.stack().reset_index()
dfA_netwerk.columns = ['source', 'target', 'weight']

for honored proposals

In [26]:
dfG_netwerk = dfG_co_occ_mtx.stack().reset_index()
dfG_netwerk.columns = ['source', 'target', 'weight']

In [27]:
#Example of selecting a specific cell of the data frame.  
#dfA_ENW_monodisci.iloc[0,10]

In [28]:
#Shape of dataframe for ENW monodiscipline proposals
#dfA_ENW_monodisci.shape

## Finding discipline from each monodiscipline proposal

In [29]:
# Pay attention to how iloc a cell selects. 'Application numbers' is zeroth column. 
dfA_ENW_mono_disciplines = pd.DataFrame(columns = ['Application numbers', 'Discipline_list', 'All_Discipline_count'])
i=0 #starts from oth row
j=1 #starts from 1st column
Aanvraag_nummer=0
k=dfA_ENW_monodisci.columns.get_loc("All_Discipline_count") #column number where the All_Discipline_count is
l=len(dfA_ENW_monodisci.index)#number of rows
for i in range (0,l):
    Discipline_count=0 #introducing zero discipline count
    Discipline_list=" " #introducing empty discipline list
    for j in range (1,k): #while the rows are not exhausted
        if (dfA_ENW_monodisci.iloc[i,j]==1) & (Discipline_count<dfA_ENW_monodisci.iloc[i,k]): #if the given cell has 1 as value
            Discipline_list=Discipline_list+ dfA_ENW_monodisci.columns[j] #adds a column name to discipline list
            Discipline_count+=1 #counts the number of disciplines with 1 as value
            if Discipline_count==dfA_ENW_monodisci.iloc[i,k]:
                Aanvraag_nummer=dfA_ENW_monodisci.iloc[i,0]
                new_row = {'Application numbers':Aanvraag_nummer, 'Discipline_list':Discipline_list, 'All_Discipline_count':dfA_ENW_monodisci.iloc[i,k]}
                dfA_ENW_mono_disciplines = dfA_ENW_mono_disciplines.append(new_row, ignore_index=True)
dfA_ENW_mono_disciplines.head(2)

/tmp/ipykernel_18700/1458031162.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfA_ENW_mono_disciplines = dfA_ENW_mono_disciplines.append(new_row, ignore_index=True)
/tmp/ipykernel_18700/1458031162.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfA_ENW_mono_disciplines = dfA_ENW_mono_disciplines.append(new_row, ignore_index=True)
/tmp/ipykernel_18700/1458031162.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfA_ENW_mono_disciplines = dfA_ENW_mono_disciplines.append(new_row, ignore_index=True)
/tmp/ipykernel_18700/1458031162.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfA_ENW_mono_disciplines = dfA_ENW_mono_dis

/tmp/ipykernel_18700/1458031162.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfA_ENW_mono_disciplines = dfA_ENW_mono_disciplines.append(new_row, ignore_index=True)
/tmp/ipykernel_18700/1458031162.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfA_ENW_mono_disciplines = dfA_ENW_mono_disciplines.append(new_row, ignore_index=True)
/tmp/ipykernel_18700/1458031162.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfA_ENW_mono_disciplines = dfA_ENW_mono_disciplines.append(new_row, ignore_index=True)
/tmp/ipykernel_18700/1458031162.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfA_ENW_mono_disciplines = dfA_ENW_mono_dis

,Application numbers,Discipline_list,All_Discipline_count
0,1,Biology,1.0
1,3,Life sciences,1.0


In [30]:
#shape (rows and columns) of the intradomein multidiscipline dataframe
#dfA_intradom_multidisci.shape

In [31]:
#Shape (rows and columns) of the dataframe
#dfA_interdom_multidisci.shape

## Finding disciplines from each intradomein multidiscipline submitted proposal

In [32]:
#The name for the dataframe with disciplines is given dfA_multi_intradom_disciplines, to differentiate from the dataframe with discipline counts
dfA_multi_intradom_disciplines = pd.DataFrame(columns = ['Application numbers', 'Discipline_list', 'All_Discipline_count'])
i=0 #starts from oth row
j=1 #starts from 1st column
Aanvraag_nummer=0
k=dfA_intradom_multidisci.columns.get_loc("All_Discipline_count")#column number where the All_Discipline_count is
l=len(dfA_intradom_multidisci.index)#number of rows
for i in range (0,l):
    Discipline_count=0 #introducing zero discipline count
    Discipline_list=" " #introducing empty discipline list
    for j in range (1,k): #while the rows are not exhausted
        if (dfA_intradom_multidisci.iloc[i,j]==1) & (Discipline_count<dfA_intradom_multidisci.iloc[i,k]): #if the given cell has 1 as value
            Discipline_list=Discipline_list+ dfA_intradom_multidisci.columns[j]+"/" #adds a column name to discipline list
            Discipline_count+=1 #counts the number of disciplines with 1 as value
            if Discipline_count==dfA_intradom_multidisci.iloc[i,k]:
                Aanvraag_nummer=dfA_intradom_multidisci.iloc[i,0]
                new_row = {'Application numbers':Aanvraag_nummer, 'Discipline_list':Discipline_list, 'All_Discipline_count':dfA_intradom_multidisci.iloc[i,k]}
                dfA_multi_intradom_disciplines = dfA_multi_intradom_disciplines.append(new_row, ignore_index=True)
                #print(discipline_list)
dfA_multi_intradom_disciplines.head(2)

/tmp/ipykernel_18700/3020201320.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfA_multi_intradom_disciplines = dfA_multi_intradom_disciplines.append(new_row, ignore_index=True)
/tmp/ipykernel_18700/3020201320.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfA_multi_intradom_disciplines = dfA_multi_intradom_disciplines.append(new_row, ignore_index=True)
/tmp/ipykernel_18700/3020201320.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfA_multi_intradom_disciplines = dfA_multi_intradom_disciplines.append(new_row, ignore_index=True)
/tmp/ipykernel_18700/3020201320.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfA_mul

,Application numbers,Discipline_list,All_Discipline_count
0,2,Biology/Life sciences/Chemistry/,3.0
1,4,Physics/Chemistry/,2.0


## Finding disciplines from each interdomein multidiscipline submitted proposal

In [33]:

dfA_multi_interdom_disciplines = pd.DataFrame(columns = ['Application numbers', 'Discipline_list', 'All_Discipline_count'])
i=0 #starts from oth row
j=1 #starts from 1st column
Aanvraag_nummer=0
k=dfA_interdom_multidisci.columns.get_loc("All_Discipline_count") #column number where the All_Discipline_count is
l=len(dfA_interdom_multidisci.index)#number of rows
for i in range (0,l):
    Discipline_count=0 #introducing zero discipline count
    Discipline_list=" " #introducing empty discipline list
    for j in range (1,k): #while the rows are not exhausted
        if (dfA_interdom_multidisci.iloc[i,j]==1) & (Discipline_count<dfA_interdom_multidisci.iloc[i,k]): #if the given cell has 1 as value
            Discipline_list=Discipline_list+ dfA_interdom_multidisci.columns[j]+"/" #adds a column name to discipline list
            Discipline_count+=1 #counts the number of disciplines with 1 as value
            if Discipline_count==dfA_interdom_multidisci.iloc[i,k]:
                Aanvraag_nummer=dfA_interdom_multidisci.iloc[i,0]
                new_row = {'Application numbers':Aanvraag_nummer, 'Discipline_list':Discipline_list, 'All_Discipline_count':dfA_interdom_multidisci.iloc[i,k]}
                dfA_multi_interdom_disciplines = dfA_multi_interdom_disciplines.append(new_row, ignore_index=True)
                #print(discipline_list)
dfA_multi_interdom_disciplines.head(2)

/tmp/ipykernel_18700/3108401138.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfA_multi_interdom_disciplines = dfA_multi_interdom_disciplines.append(new_row, ignore_index=True)
/tmp/ipykernel_18700/3108401138.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfA_multi_interdom_disciplines = dfA_multi_interdom_disciplines.append(new_row, ignore_index=True)
/tmp/ipykernel_18700/3108401138.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfA_multi_interdom_disciplines = dfA_multi_interdom_disciplines.append(new_row, ignore_index=True)
/tmp/ipykernel_18700/3108401138.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfA_mul

,Application numbers,Discipline_list,All_Discipline_count
0,5,Medicine/Life sciences/,2.0
1,14,Biology/Information Technology/Technical Scie...,3.0


In [34]:
#Shape (rows and columns) of the dataframe
#dfG_ENW_monodisci.shape

## Finding disciplines from each monodiscipline Successful proposal

In [35]:
dfG_ENW_mono_disciplines = pd.DataFrame(columns = ['Application numbers', 'Discipline_list', 'All_Discipline_count'])
i=0 #starts from oth row
j=1 #starts from 1st column
Aanvraag_nummer=0
k=dfG_ENW_monodisci.columns.get_loc("All_Discipline_count") #column number where the All_Discipline_count is
l=len(dfG_ENW_monodisci.index)#number of rows
for i in range (0,l):
    Discipline_count=0 #introducing zero discipline count
    Discipline_list=" " #introducing empty discipline list
    for j in range (1,k): #while the rows are not exhausted
        if (dfG_ENW_monodisci.iloc[i,j]==1) & (Discipline_count<dfG_ENW_monodisci.iloc[i,k]): #if the given cell has 1 as value
            Discipline_list=Discipline_list+ dfG_ENW_monodisci.columns[j] #adds a column name to discipline list
            Discipline_count+=1 #counts the number of disciplines with 1 as value
            if Discipline_count==dfG_ENW_monodisci.iloc[i,k]:
                Aanvraag_nummer=dfG_ENW_monodisci.iloc[i,0]
                new_row = {'Application numbers':Aanvraag_nummer, 'Discipline_list':Discipline_list, 'All_Discipline_count':dfG_ENW_monodisci.iloc[i,k]}
                dfG_ENW_mono_disciplines = dfG_ENW_mono_disciplines.append(new_row, ignore_index=True)
                #print(discipline_list)
dfG_ENW_mono_disciplines.head(2)

/tmp/ipykernel_18700/1205715904.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfG_ENW_mono_disciplines = dfG_ENW_mono_disciplines.append(new_row, ignore_index=True)
/tmp/ipykernel_18700/1205715904.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfG_ENW_mono_disciplines = dfG_ENW_mono_disciplines.append(new_row, ignore_index=True)
/tmp/ipykernel_18700/1205715904.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfG_ENW_mono_disciplines = dfG_ENW_mono_disciplines.append(new_row, ignore_index=True)
/tmp/ipykernel_18700/1205715904.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfG_ENW_mono_disciplines = dfG_ENW_mono_dis

,Application numbers,Discipline_list,All_Discipline_count
0,15,Mathematics,1.0
1,17,Life sciences,1.0


In [36]:
#Shape (rows and columns) of the intradomei multidiscipline honored dataframe
#dfG_intradom_multidisci.shape

## Finding disciplines from each intradomein multidiscipline proposal which were successful getting grant

In [37]:

#The dataframe with disciplines is called dfA_multi_intradom_disciplines, to differentiate from the dataframe with discipline counts
dfG_multi_intradom_disciplines = pd.DataFrame(columns = ['Application numbers', 'Discipline_list', 'All_Discipline_count'])
i=0 #starts from oth row
j=1 #starts from 1st column
Aanvraag_nummer=0
k=dfG_intradom_multidisci.columns.get_loc("All_Discipline_count") #column number where the All_Discipline_count is
l=len(dfG_intradom_multidisci.index)#number of rows
for i in range (0,l):
    Discipline_count=0 #introducing zero discipline count
    Discipline_list=" " #introducing empty discipline list
    for j in range (1,k): #while the rows are not exhausted
        if (dfG_intradom_multidisci.iloc[i,j]==1) & (Discipline_count<dfG_intradom_multidisci.iloc[i,k]): #if the given cell has 1 as value
            Discipline_list=Discipline_list+ dfG_intradom_multidisci.columns[j]+"/" #adds a column name to discipline list
            Discipline_count+=1 #counts the number of disciplines with 1 as value
            if Discipline_count==dfG_intradom_multidisci.iloc[i,k]:
                Aanvraag_nummer=dfG_intradom_multidisci.iloc[i,0]
                new_row = {'Application numbers':Aanvraag_nummer, 'Discipline_list':Discipline_list, 'All_Discipline_count':dfG_intradom_multidisci.iloc[i,k]}
                dfG_multi_intradom_disciplines = dfG_multi_intradom_disciplines.append(new_row, ignore_index=True)
                #print(discipline_list)
dfG_multi_intradom_disciplines.head(2)

/tmp/ipykernel_18700/1745920468.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfG_multi_intradom_disciplines = dfG_multi_intradom_disciplines.append(new_row, ignore_index=True)
/tmp/ipykernel_18700/1745920468.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfG_multi_intradom_disciplines = dfG_multi_intradom_disciplines.append(new_row, ignore_index=True)
/tmp/ipykernel_18700/1745920468.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfG_multi_intradom_disciplines = dfG_multi_intradom_disciplines.append(new_row, ignore_index=True)
/tmp/ipykernel_18700/1745920468.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfG_mul

,Application numbers,Discipline_list,All_Discipline_count
0,4,Physics/Chemistry/,2.0
1,12,Life sciences/Physics/,2.0


In [38]:
#Shape (rows and columns) of the dataframe
#dfG_interdom_multidisci.shape

## Finding disciplines from each multi discipline interdomein successful proposal contains

In [39]:
dfG_multi_interdom_disciplines = pd.DataFrame(columns = ['Application numbers', 'Discipline_list', 'All_Discipline_count'])
i=0 #starts from oth row
j=1 #starts from 1st column
Aanvraag_nummer=0
k=dfG_interdom_multidisci.columns.get_loc("All_Discipline_count")  #column number where the All_Discipline_count is
l=len(dfG_interdom_multidisci.index)#number of rows
for i in range (0,l):
    Discipline_count=0 #introducing zero discipline count
    Discipline_list=" " #introducing empty discipline list
    for j in range (1,k): #while the rows are not exhausted
        if (dfG_interdom_multidisci.iloc[i,j]==1) & (Discipline_count<dfG_interdom_multidisci.iloc[i,k]): #if the given cell has 1 as value
            Discipline_list=Discipline_list+ dfG_interdom_multidisci.columns[j]+"/" #adds a column name to discipline list
            Discipline_count+=1 #counts the number of disciplines with 1 as value
            if Discipline_count==dfG_interdom_multidisci.iloc[i,k]:
                Aanvraag_nummer=dfG_interdom_multidisci.iloc[i,0]
                new_row = {'Application numbers':Aanvraag_nummer, 'Discipline_list':Discipline_list, 'All_Discipline_count':dfG_interdom_multidisci.iloc[i,k]}
                dfG_multi_interdom_disciplines = dfG_multi_interdom_disciplines.append(new_row, ignore_index=True)
                #print(discipline_list)
dfG_multi_interdom_disciplines.head(2)

/tmp/ipykernel_18700/944932687.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfG_multi_interdom_disciplines = dfG_multi_interdom_disciplines.append(new_row, ignore_index=True)


,Application numbers,Discipline_list,All_Discipline_count
0,79,Biology/Medicine/Life sciences/Mathematics/,4.0


## monodiscipline proposals

reindexing monodiscipline dataframes with Disciplines and number of counts as columns

In [41]:
dfA_mono_reindex=dfA_ENW_mono_disciplines.Discipline_list.value_counts().reset_index().rename(columns={'index': 'Disciplines', 0: 'count'})
dfG_mono_reindex=dfG_ENW_mono_disciplines.Discipline_list.value_counts().reset_index().rename(columns={'index': 'Disciplines', 0: 'count'})

Creating summary dataframe of monodiscipline proposals

In [42]:
#Merging the submitted and the successful monodiscipline proposals dataframes
df_summary_monodisci=pd.merge(dfA_mono_reindex, dfG_mono_reindex, on="Disciplines", how="left")
# Rename the existing DataFrame (rather than creating a copy) 
df_summary_monodisci.rename(columns={'Discipline_list_x': 'Applied', 'Discipline_list_y': 'Successful'}, inplace=True)

#adding a column of % Succes
df_summary_monodisci['% Succes']=df_summary_monodisci['Successful'].div(df_summary_monodisci['Applied'].values, axis=0)
df_summary_monodisci['% Succes']=df_summary_monodisci['% Succes'].mul(100).round(1)
df_summary_monodisci=df_summary_monodisci.fillna(0)

## multidiscipline intradomein proposals

reindexing multidiscipline intradomein dataframes with Disciplines and number of counts as columns

In [43]:
dfA_intradom_multidisci_reindex=dfA_multi_intradom_disciplines.Discipline_list.value_counts().reset_index().rename(columns={'index': 'Disciplines', 0: 'count'})
dfG_intradom_multidisci_reindex=dfG_multi_intradom_disciplines.Discipline_list.value_counts().reset_index().rename(columns={'index': 'Disciplines', 0: 'count'})

Creating summary dataframe of multidiscipline intradomein proposals

In [44]:
#Merging the submitted and the successful multidiscipline intradomein proposals dataframes
df_summary_intradom_multidisci=pd.merge(dfA_intradom_multidisci_reindex, dfG_intradom_multidisci_reindex, on="Disciplines", how="left")
# Rename the existing DataFrame (rather than creating a copy) 
df_summary_intradom_multidisci.rename(columns={'Discipline_list_x': 'Applied', 'Discipline_list_y': 'Successful'}, inplace=True)

#adding a column of % Succes
df_summary_intradom_multidisci['% Succes']=df_summary_intradom_multidisci['Successful'].div(df_summary_intradom_multidisci['Applied'].values, axis=0)
df_summary_intradom_multidisci['% Succes']=df_summary_intradom_multidisci['% Succes'].mul(100).round(1)
df_summary_intradom_multidisci=df_summary_intradom_multidisci.fillna(0)

## multidiscipline  Interdomein  proposals:

reindexing multidiscipline interdomein dataframes with Disciplines and number of counts as columns

In [45]:
dfA_interdom_multidisci_reindex=dfA_multi_interdom_disciplines.Discipline_list.value_counts().reset_index().rename(columns={'index': 'Disciplines', 0: 'count'})
dfG_interdom_multidisci_reindex=dfG_multi_interdom_disciplines.Discipline_list.value_counts().reset_index().rename(columns={'index': 'Disciplines', 0: 'count'})

Creating summary dataframe of multidiscipline interdomein proposals

In [46]:
#Merging the submitted and the successful multidiscipline interdomein proposals dataframes
df_summary_interdom_multidisci=pd.merge(dfA_interdom_multidisci_reindex, dfG_interdom_multidisci_reindex, on="Disciplines", how="left")
# Rename the existing DataFrame (rather than creating a copy) 
df_summary_interdom_multidisci.rename(columns={'Discipline_list_x': 'Applied', 'Discipline_list_y': 'Successful'}, inplace=True)

#adding a column of % Succes
df_summary_interdom_multidisci['% Succes']=df_summary_interdom_multidisci['Successful'].div(df_summary_interdom_multidisci['Applied'].values, axis=0)
df_summary_interdom_multidisci['% Succes']=df_summary_interdom_multidisci['% Succes'].mul(100).round(1)
df_summary_interdom_multidisci=df_summary_interdom_multidisci.fillna(0)

# Output with Openpyxl

The openpyxl.utils.dataframe.dataframe_to_rows() function provides a simple way to work with Pandas Dataframes

In [47]:
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import Workbook
from openpyxl.chart import BarChart, Series, Reference

In [48]:
wb = Workbook()
ws = wb.active
#ws = wb.create_sheet("Samenvatting", 0) 
ws.title='Summary'

for r in dataframe_to_rows(df_summary, index=False, header=True):
    ws.append(r)

for cell in ws['A'] + ws[1]:
    cell.style = 'Pandas'

chart0 = BarChart()
chart0.type = "col"
chart0.style = 10
chart0.title = "Summary"
chart0.y_axis.title = 'Proposals'
chart0.x_axis.title = ''

data = Reference(ws, min_col=2, min_row=1, max_row=6, max_col=3)
cats = Reference(ws, min_col=1, min_row=2, max_row=6)
chart0.add_data(data, titles_from_data=True)
chart0.set_categories(cats)
chart0.shape = 20
ws.add_chart(chart0, "F1")

writing summary of analysis for monodisci, intradom_multidisci, interdom_multidisci dataframes in subsequent excel sheets

In [49]:
ws1 = wb.create_sheet("Monodiscipline", 1) 
for r in dataframe_to_rows(df_summary_monodisci, index=False, header=True):
    ws1.append(r)

chart1 = BarChart()
chart1.type = "col"
chart1.style = 10
chart1.title = "Monodiscipline Character"
chart1.y_axis.title = 'Proposals'
chart1.x_axis.title = ''

data = Reference(ws1, min_col=2, min_row=1, max_row=(len(df_summary_monodisci.index)+1), max_col=3)
cats = Reference(ws1, min_col=1, min_row=2, max_row=(len(df_summary_monodisci.index)+1))
chart1.add_data(data, titles_from_data=True)
chart1.set_categories(cats)
chart1.shape = 20
ws1.add_chart(chart1, "F1")
    
ws2 = wb.create_sheet("Intradomein multidisciplinaire", 2)
for r in dataframe_to_rows(df_summary_intradom_multidisci, index=False, header=True):
    ws2.append(r)

chart2 = BarChart()
chart2.type = "col"
chart2.style = 10
chart2.title = "Intradomein multidisciplinaire Character"
chart2.y_axis.title = 'Proposals'
chart2.x_axis.title = ''

data = Reference(ws2, min_col=2, min_row=1, max_row=(len(df_summary_intradom_multidisci.index)+1), max_col=3)
cats = Reference(ws2, min_col=1, min_row=2, max_row=(len(df_summary_intradom_multidisci.index)+1))
chart2.add_data(data, titles_from_data=True)
chart2.set_categories(cats)
chart2.shape = 40
ws2.add_chart(chart2, "F1")

ws3 = wb.create_sheet("Interdomein multidisciplinaire", 3)
for r in dataframe_to_rows(df_summary_interdom_multidisci, index=False, header=True):
    ws3.append(r)
    
chart3 = BarChart()
chart3.type = "col"
chart3.style = 10
chart3.title = "Interdomein multidisciplinaire Karakter"
chart3.y_axis.title = 'Proposals'
chart3.x_axis.title = ''

data = Reference(ws3, min_col=2, min_row=1, max_row=(len(df_summary_interdom_multidisci.index)+1), max_col=3)
cats = Reference(ws3, min_col=1, min_row=2, max_row=(len(df_summary_interdom_multidisci.index)+1))
chart3.add_data(data, titles_from_data=True)
chart3.set_categories(cats)
chart3.shape = 80
ws3.add_chart(chart3, "F1")

Writing a co-occurance matrix and file for VOS viewer for submitted proposals

In [50]:
ws4=wb.create_sheet("AP Co occurance matrix", 4)
for r in dataframe_to_rows(dfA_co_occ_mtx, index=True, header=True):
    ws4.append(r)

ws5=wb.create_sheet("AP Network file VOS viewer", 5)
for r in dataframe_to_rows(dfA_netwerk, index=False, header=True):
    ws5.append(r)

Writing a co-occurance matrix and file for VOS viewer for Successful proposals

In [51]:
ws6=wb.create_sheet("SUC Co occurance matrix", 6)
for r in dataframe_to_rows(dfG_co_occ_mtx, index=True, header=True):
    ws6.append(r)

ws7=wb.create_sheet("SUC Network file VOS viewer", 7)
for r in dataframe_to_rows(dfG_netwerk, index=False, header=True):
    ws7.append(r)

saving the excel workbook

In [52]:
# write the name of the file with extension .xlsx
Exportfile_name=input('Give export file name with extension .xlsx ',)
wb.save(Exportfile_name)

Give export file name with extension .xlsx discipline_analysis.xlsx
